# out-of-core image analysis with dask

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
%gui qt

In [ ]:
# needed for %gui qt to work in jupyter book

import time
time.sleep(5)
del time

In [ ]:
import dask.array as da

(Note: in the first pass, don't call the next cell. Then we will rerun the notebook, seeing the benefits of these cells.)

In [ ]:
from dask import distributed
client = distributed.Client()
print(client.dashboard_link)

In [ ]:
random_image = np.random.random((512, 512))

import napari

napari.view_image(random_image)

In [ ]:
impossible_image = da.random.random(
    (20_000, 8_000, 8_000),
    chunks=(1, 4_000, 4_000),
)

print(impossible_image.nbytes / 1e9)

In [ ]:
napari.view_image(impossible_image)

In [ ]:
from dask_image.imread import imread

In [ ]:
embryo = imread('/Users/jni/data/Fluo-N3DH-CE/01/t*.tif')

In [ ]:
type(embryo)

In [ ]:
embryo.shape

In [ ]:
embryo.dtype

In [ ]:
embryo.nbytes / 1e9

In [ ]:
embryo.chunksize

In [ ]:
viewer = napari.view_image(
    embryo,
    scale=[10, 1, 1],
)

In [ ]:
import dask

In [ ]:
from dask_image import ndfilters

smoothed = ndfilters.gaussian_filter(
    embryo,
    sigma=[0, 1, 10, 10],
)

smoothed_layer = viewer.add_image(
    smoothed,
    scale=[10, 1, 1],
)

In [ ]:
from skimage.morphology import h_maxima

In [ ]:
maxes = dask.compute(np.max(smoothed, axis=(1, 2, 3)))[0]  # takes a minute or so, check dashboard!

In [ ]:
plt.plot(maxes)

In [ ]:
result0 = np.transpose(np.nonzero(h_maxima(np.array(smoothed[0]), 90)))
result0.shape

In [ ]:
# this function needs dramatic improvement

def peaks_from_dask(idx, volume, min_height=90):
    arr = np.asarray(volume)
    maxima = h_maxima(arr, min_height)
    coords = np.transpose(np.nonzero(maxima))
    ncoord = coords.shape[0]
    coords = np.concatenate(
        (np.full((ncoord, 1), idx), coords),
        axis=1,
    )
    return coords

In [ ]:
futures = client.map(
    peaks_from_dask,
    np.arange(smoothed.shape[0]),
    smoothed,
)

In [ ]:
for f in futures:
    f.cancel()

In [ ]:
futures_small = client.map(peaks_from_dask, np.arange(10), smoothed[:10])

In [ ]:
coords = np.concatenate(
    [f.result() for f in futures_small],
    axis=0,
)

In [ ]:
viewer.add_points(
    coords * [1, 10, 1, 1],
    name='coords',
)

In [ ]:
coords[:10]